In [1]:
import pandas as pd
import numpy as np
from time import time

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.svm import SVC

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# ***TASK 1***

## Read data

In [2]:
df2 = pd.read_csv('./data/Task1_2.csv', sep=';')
print(df2.shape)

(4070, 9)


In [3]:
df2.head()

,POUG,TRE,ID,ZUB,VOL,UIO,VBNM,Type,OIN
0,1,1.750,0,t,f,uuuu,t,n,17.92
1,0,0.290,1,f,f,wwww,f,n,16.92
2,1,0.000,2,f,f,wwww,t,n,31.25
3,0,0.335,3,f,f,uuuu,f,n,48.17
4,0,0.500,4,t,f,wwww,f,n,32.33


In [4]:
df1 = pd.read_csv('./data/Task1_1.csv', sep=';')
print(df1.shape)

(4070, 11)


In [5]:
df1.head()

,ID,UKL,GJAH,ZIK,HUI,ERZ,CDx,BJZHD,NKJUD,LPI,BJKG
0,0,160,oooo,x,oooo,www,5.0,vvvv,80.0,800000.0,qqqq
1,1,153,rrr,NaN,uuu,pppp,0.0,mmm,200.0,2000000.0,qqqq
2,2,5,oooo,x,oooo,www,19.0,hh,96.0,960000.0,hh
3,3,9,oooo,NaN,oooo,www,120.0,kkk,0.0,0.0,qqq
4,4,40,oooo,y,oooo,www,0.0,mmm,232.0,2320000.0,qqqq


## Preprocess & EDA

### Drop duplicates

Prior to merging `df1` and `df2` we need to ensure unique records. We see below there are 370 duplicate rows, where "duplicate" is defined as having the same value for all columns in a row.

In [6]:
print(f"Number of duplicates in df1: {df1.duplicated().sum()}")
print(f"Number of duplicates in df2: {df2.duplicated().sum()}")

Number of duplicates in df1: 370
Number of duplicates in df2: 370


In [7]:
df1 = df1.drop_duplicates().reset_index(drop=True)
df2 = df2.drop_duplicates().reset_index(drop=True)

assert df1.duplicated().sum() == 0  # sanity check
assert df2.duplicated().sum() == 0

print(df1.shape)
print(df2.shape)

(3700, 11)
(3700, 9)


### Merge

Although it's not explicitly stated in the instructions, I assume an inner join is desired. As we can see, there's a 100% match rate.

In [8]:
df = df1.merge(df2, on='ID', how='inner')
print(df.shape)

(3700, 19)


In [9]:
df.head()

,ID,UKL,GJAH,ZIK,HUI,ERZ,CDx,BJZHD,NKJUD,LPI,BJKG,POUG,TRE,ZUB,VOL,UIO,VBNM,Type,OIN
0,0,160,oooo,x,oooo,www,5.0,vvvv,80.0,800000.0,qqqq,1,1.750,t,f,uuuu,t,n,17.92
1,1,153,rrr,NaN,uuu,pppp,0.0,mmm,200.0,2000000.0,qqqq,0,0.290,f,f,wwww,f,n,16.92
2,2,5,oooo,x,oooo,www,19.0,hh,96.0,960000.0,hh,1,0.000,f,f,wwww,t,n,31.25
3,3,9,oooo,NaN,oooo,www,120.0,kkk,0.0,0.0,qqq,0,0.335,f,f,uuuu,f,n,48.17
4,4,40,oooo,y,oooo,www,0.0,mmm,232.0,2320000.0,qqqq,0,0.500,t,f,wwww,f,n,32.33


In [10]:
# Drop ID column since no longer needed for modeling
del df['ID']

### Class imbalance

As the instructions suggested - and the data below confirms - this is a binary classification problem. Importantly, the target variable is highly class imbalanced (i.e. the distribution of classes is highly unequal), with ~92.5% of cases being "y", while only ~7.5% being "n". While phenomenon is common in real-world applications, it also poses some modeling challenges.

To address this, I focus on performance metrics that differentiate between performance by class (e.g. precision, recall, F1 score, confusion matrix), rather than "global" performance indicators like accuracy. The reason being that in expectation, a model could be (in this case) about 92.5% accurate simply by predicting the dominant class for every observation, which would be a poor model.

In [11]:
df['Type'].value_counts(normalize=True)

y    0.925405
n    0.074595
Name: Type, dtype: float64

### Missing data

As we see below, several of our predictor features contain missing data (though not our target feature). Because I use an ensemble method below that averages predictions across three different ML models, two of which cannot easily handle missing data (SVM & random forest) I impute these missing values.

Several imputation strategies exist for missing data, including:
- listwise deletion - drop rows with any missing column values
- unconditional imputation - use measure of central tendency (mean or median) among non-missing rows
- conditional imputation - use e.g. a ML model to first impute the data iteratively by feature, using all other features

While conditional imputation offers the best performance in expectation, for sake of time I use an unconditional imputation method.

**For our modeling purposes, imputation is importantly only required for numeric features**, not categorical features. This is because we'll one-hot encode categorical features, which enables us to code "missing" as simply another feature value. 

In [12]:
# Share of missing data by column
df.isnull().mean()

UKL      0.000000
GJAH     0.017297
ZIK      0.579730
HUI      0.000000
ERZ      0.017297
CDx      0.000000
BJZHD    0.017838
NKJUD    0.027027
LPI      0.027027
BJKG     0.017838
POUG     0.000000
TRE      0.000000
ZUB      0.000000
VOL      0.000000
UIO      0.010541
VBNM     0.000000
Type     0.000000
OIN      0.010541
dtype: float64

In [13]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove('POUG')  # this appears categorical so we remove

In [14]:
# Statistical moments before imputation
df[numeric_cols].describe()

,UKL,CDx,NKJUD,LPI,TRE,OIN
count,3700.000000,3700.000000,3600.000000,3.600000e+03,3700.000000,3661.000000
mean,95.688378,2246.705946,162.695000,1.626950e+06,3.439496,32.820713
std,56.382436,8708.571126,156.045682,1.560457e+06,4.335229,12.666181
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000,13.750000
25%,46.000000,0.000000,0.000000,0.000000e+00,0.500000,23.000000
50%,99.000000,113.000000,120.000000,1.200000e+06,1.750000,28.670000
75%,152.000000,1059.750000,280.000000,2.800000e+06,5.000000,40.830000
max,179.000000,100000.000000,1160.000000,1.160000e+07,28.500000,80.250000


In [15]:
for col in numeric_cols:
    mu = df[df[col].notnull()][col].mean()  # mean as measure of central tendency
    df[col] = np.where(df[col].isnull(), mu, df[col])

In [16]:
assert df[numeric_cols].isnull().mean().sum() == 0

In [17]:
# Verify statistical moments appear similar after imputation
df[numeric_cols].describe()

,UKL,CDx,NKJUD,LPI,TRE,OIN
count,3700.000000,3700.000000,3700.000000,3.700000e+03,3700.000000,3700.000000
mean,95.688378,2246.705946,162.695000,1.626950e+06,3.439496,32.820713
std,56.382436,8708.571126,153.921934,1.539219e+06,4.335229,12.599232
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000,13.750000
25%,46.000000,0.000000,0.000000,0.000000e+00,0.500000,23.000000
50%,99.000000,113.000000,120.000000,1.200000e+06,1.750000,28.670000
75%,152.000000,1059.750000,274.000000,2.740000e+06,5.000000,40.000000
max,179.000000,100000.000000,1160.000000,1.160000e+07,28.500000,80.250000


### One-hot encode categorical features

One hot encoding a feature involves creating a new boolean feature to typically represent each unique value in source feature. To do this we use sklearn's `OneHotEncoder` class. Depending on the model (e.g. linear regression or regression using maximum likelihood without regularization), it's necessary to drop one category value, otherwise the one-hot columns will be perfectly collinear. However, collinearity is typically not a problem for machine learning models.

In [18]:
categorical_cols = [i for i in df.columns if i not in numeric_cols and "Type" not in i]  # exclude target

In [19]:
# Number of unique values among categorical features
df[categorical_cols].nunique(dropna=False)

GJAH      4
ZIK       3
HUI       3
ERZ       4
BJZHD    13
BJKG      9
POUG     23
ZUB       2
VOL       2
UIO       3
VBNM      2
dtype: int64

In [20]:
X = OneHotEncoder().fit_transform(df[categorical_cols]).toarray()
assert (df[categorical_cols].nunique(dropna=False).sum() == X.shape[1])  # ensure yields full number of unique values

#### Concatenate numeric features 

To complete our `X` matrix of predictor features, we concatenate the numeric features to our newly created array of one-hot features

In [21]:
X = np.concatenate((X, df[numeric_cols].values), axis=1)

### Label encode target feature

Label encoding is the process of converting the non-numeric values of a categorical feature to a numeric representation. 

**Given the class imbalance described above, I recode the rarer "n" category to 1, and the more common "y" category to 0.** This coding procedure is common when modeling using imbalanced data, allow us to focus on classical measures of precision and recall, which assume the user is interested in the harder-to-predict class labeled 1.

In [22]:
y = df['Type'].replace({'n': 1, 'y': 0}).values

### Train-test split

I split the data into train and test/holdout subsets, allowing me to evaluate the trained model performance on the unseen test set. Although this split is admittedly somewhat arbitrary since we have no "true" test set in this case, it nonetheless provides a straightforward way to evaluate model performance and compare across models on unseen data. 

*Note* - I use k-fold cross-validation below, so I do not create a traditional cross-validation set.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

print(X_train.shape)
print(X_test.shape)

(3330, 74)
(370, 74)


### Data normalization

We use sklearn's `MinMaxScaler`, which transforms each of our predictor features to 0-1 scale. Data normalization is used to convert the range of features to a similar scale, which speeds up training. An alternative method would be standardization (i.e. z-score normalization; mean of 0, std of 1).

**Importantly normalization takes place *after* splitting the data into train and test sets**. This ensures not only that there is no data leakage between subsets, but also that the transformed distributions are consistent between subsets (e.g. all range from 0-1, since they're using the minimum and maximum values of that subset). 

In [24]:
scaler = MinMaxScaler()
X_train_ = scaler.fit_transform(X_train)
X_test_ = scaler.fit_transform(X_test)

## Modelling

### Performance metrics

In [25]:
def metrics(y_true, y_pred):
    '''
    Calculates binary classification performance metrics for a given model.
    :param y_true: array_like, truth values as int
    :param y_pred: array_like, predicted values as int
    :returns: dict, with keys for each metric: 
        accuracy - proportion of correct predictions out of total predictions
        sensitivity - (aka recall), of all true positives how many did we correctly predict as positive
        specificity - (aka selectivity/TNR), of all true negatives how many did we correctly predict as negative
        precision - of all predicted positive cases how many were actually positive
        F-1 score - harmonic/weighted mean of precision and sensitivity scores
        ROC-AUC - area under receiver operating characteristic curve
        
    '''
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    metrics_dict = {}
    rounding = 6
    metrics_dict['accuracy'] = round((tp + tn) / len(y_true), rounding)
    metrics_dict['sensitivity'] = round(tp / (fn + tp), rounding) # aka recall
    metrics_dict['specificity'] = round(tn / (tn + fp), rounding) # aka TNR
    metrics_dict['precision'] = round(tp / (tp + fp), rounding)
    metrics_dict['f1'] = round(2 * (metrics_dict['precision'] * metrics_dict['sensitivity']) \
                        / (metrics_dict['precision'] + metrics_dict['sensitivity']), rounding)
    metrics_dict['roc_auc'] = round(roc_auc_score(y_true, y_pred), rounding)
    
    return metrics_dict

#### Support vector machines (SVM) 

SVM is a type of supervised ML algorithm use for classification and regression. In support vector classification, the model finds a hyperplane in n-dimensional space that maximizes the distance (margin) between classes. 

In [26]:
svc = SVC(probability=True)
svc.fit(X_train_, y_train)
pred = svc.predict(X_test_)
probab = svc.predict_proba(X_test_)

In [27]:
metrics(y_test, pred)

{'accuracy': 0.986486,
 'sensitivity': 0.782609,
 'specificity': 1.0,
 'precision': 1.0,
 'f1': 0.878049,
 'roc_auc': 0.891304}